### Install necessary packages

In [1]:
!pip install tensorflow keras scikit-fuzzy deap


[notice] A new release of pip available: 22.3.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import numpy as np
import skfuzzy as fuzz
from tensorflow.keras.preprocessing import image
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from deap import base, creator, tools, algorithms
import random
import matplotlib.pyplot as plt
import tensorflow as tf

### Data augmentation for training set

In [3]:
train_datagen = ImageDataGenerator(
    rescale=1./255,               
    shear_range=0.2,             
    zoom_range=0.2,               
    horizontal_flip=True          
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    "C:\\Users\\janardhan\\Documents\\Softcomputing Lab\\Soft Computing Project\\dataset\\Train",         
    target_size=(224, 224),       
    batch_size=32,                
    class_mode='binary'           
)

test_generator = test_datagen.flow_from_directory(
    "C:\\Users\\janardhan\\Documents\\Softcomputing Lab\\Soft Computing Project\\dataset\\Test",       
    target_size=(224, 224),       
    batch_size=32,                
    class_mode='binary'           
)


Found 5672 images belonging to 2 classes.
Found 1779 images belonging to 2 classes.


### CNN model architecture and Compile the Model

In [4]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary output: fresh (1) or rotten (0)
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


### Train the model

In [5]:
history = model.fit(
    train_generator,          
    epochs=10,                
    validation_data=test_generator  
)

Epoch 1/10
178/178 [==============================] - 324s 2s/step - loss: 0.3816 - accuracy: 0.8389 - val_loss: 0.2264 - val_accuracy: 0.8960
Epoch 2/10
178/178 [==============================] - 236s 1s/step - loss: 0.2336 - accuracy: 0.9051 - val_loss: 0.1860 - val_accuracy: 0.9297
Epoch 3/10
178/178 [==============================] - 231s 1s/step - loss: 0.1906 - accuracy: 0.9256 - val_loss: 0.2575 - val_accuracy: 0.8943
Epoch 4/10
178/178 [==============================] - 258s 1s/step - loss: 0.1861 - accuracy: 0.9230 - val_loss: 0.1990 - val_accuracy: 0.9095
Epoch 5/10
178/178 [==============================] - 243s 1s/step - loss: 0.1591 - accuracy: 0.9346 - val_loss: 0.1566 - val_accuracy: 0.9325
Epoch 6/10
178/178 [==============================] - 353s 2s/step - loss: 0.1486 - accuracy: 0.9457 - val_loss: 0.1535 - val_accuracy: 0.9399
Epoch 7/10
178/178 [==============================] - 366s 2s/step - loss: 0.1259 - accuracy: 0.9512 - val_loss: 0.1049 - val_accuracy: 0.9584

### Final test accuracy

In [6]:
test_loss, test_accuracy = model.evaluate(test_generator)
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

56/56 [==============================] - 17s 302ms/step - loss: 0.0511 - accuracy: 0.9792
Test Accuracy: 97.92%


### Fuzzy logic decision after CNN prediction

In [7]:
def fuzzy_logic_decision(cnn_output, fresh_params, rotten_params):
    fresh_params = np.sort(fresh_params)
    rotten_params = np.sort(rotten_params)
    
    fresh_membership = fuzz.trimf(np.arange(0, 1.1, 0.1), fresh_params)
    rotten_membership = fuzz.trimf(np.arange(0, 1.1, 0.1), rotten_params)
    
    fresh_degree = fuzz.interp_membership(np.arange(0, 1.1, 0.1), fresh_membership, cnn_output)
    rotten_degree = fuzz.interp_membership(np.arange(0, 1.1, 0.1), rotten_membership, cnn_output)
    print(f"Fresh Degree: {fresh_degree}")
    print(f"Rotten Degree: {rotten_degree}")
    
    if fresh_degree > rotten_degree:
        return "Fresh"
    else:
        return "Rotten"

### CNN output for a test image

In [20]:
img_path = "fresh4.png" 
test_image = image.load_img(img_path, target_size=(224, 224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis=0)
test_image /= 255.0

cnn_output = model.predict(test_image)[0][0]
print("CNN Output Probability (Fresh):", cnn_output)

1/1 [==============================] - 0s 42ms/step
CNN Output Probability (Fresh): 0.6556906


### Genetic Algorithm

In [13]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

# Initialize individual with random fuzzy set parameters [a, b, c] for "Low", "Medium", "High" for Fresh and Rotten
def create_individual():
    fresh_params = [np.random.uniform(0.4, 0.7), np.random.uniform(0.7, 0.9), np.random.uniform(0.9, 1.0)]  # Fresh params
    rotten_params = [np.random.uniform(0.0, 0.1), np.random.uniform(0.1, 0.3), np.random.uniform(0.3, 0.5)]  # Rotten params
    return fresh_params + rotten_params

def evaluate(individual, cnn_output):
    fresh_params = individual[:3]
    rotten_params = individual[3:]
    
    decision = fuzzy_logic_decision(cnn_output, fresh_params, rotten_params)
    
    if cnn_output > 0.5 and decision == "Fresh":
        return (1.0,)
    elif cnn_output <= 0.5 and decision == "Rotten":
        return (1.0,)
    else:
        return (0.0,)

toolbox = base.Toolbox()
toolbox.register("individual", tools.initIterate, creator.Individual, create_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=0.1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate, cnn_output=cnn_output)

population = toolbox.population(n=10)
algorithms.eaSimple(population, toolbox, cxpb=0.7, mutpb=0.2, ngen=20, verbose=True)

best_individual = tools.selBest(population, 1)[0]
print("Best Fuzzy Membership Parameters (Fresh and Rotten):", best_individual)

final_decision = fuzzy_logic_decision(cnn_output, best_individual[:3], best_individual[3:])
print(f"Final Fuzzy Logic Decision: {final_decision}")


Fresh Degree: 0.38226767643042414
Rotten Degree: 0.0
Fresh Degree: 0.48595674012088824
Rotten Degree: 0.0
Fresh Degree: 0.35512852050673194
Rotten Degree: 0.0
Fresh Degree: 0.1943741663495993
Rotten Degree: 0.0
Fresh Degree: 0.21741368960820417
Rotten Degree: 0.0
Fresh Degree: 0.2176938081908106
Rotten Degree: 0.0
Fresh Degree: 0.6598706419500061
Rotten Degree: 0.0
Fresh Degree: 0.5028513755817474
Rotten Degree: 0.0
Fresh Degree: 0.361021396225666
Rotten Degree: 0.0
Fresh Degree: 0.13875401758910197
Rotten Degree: 0.0
gen	nevals
0  	10    
Fresh Degree: 0.13875401758910197
Rotten Degree: 0.0
Fresh Degree: 0.38226767643042414
Rotten Degree: 0.0
1  	2     
Fresh Degree: 0.431613617823118
Rotten Degree: 0.0
Fresh Degree: 0.21002503412387663
Rotten Degree: 0.0
Fresh Degree: 0.5049047528270978
Rotten Degree: 0.0
Fresh Degree: 0.21187367393585743
Rotten Degree: 0.0
Fresh Degree: 0.19417333825416297
Rotten Degree: 0.0
Fresh Degree: 0.37042033125569457
Rotten Degree: 0.0
Fresh Degree: 0.567369